In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
#df = pd.read_csv (r'C:\Notebook scripts\Features_Variant_1.csv')
df = pd.read_csv ('Features_Variant_1.csv')

data_train=np.asarray(df.iloc[:,:])
#print(np.shape(data_train))

#shuffle before folding
np.random.shuffle(data_train)

In [3]:
#fold_1, fold_2, fold_3, fold_4, fold_5= np.array_split(data_train,5)
data_train_arr=np.array_split(data_train,5) #{[],[],[],[],[]}

y_train_arr=[]
x_train_arr=[]
for i in range(len(data_train_arr)):
    y_train_arr.append(data_train_arr[i][:,-1].reshape((-1, 1)) )
    x_train_arr.append(data_train_arr[i][:,:-1])


In [4]:
#потом нормализовать по ф-ле
mean_item=[]
std_item=[]
#print(x_train_arr[0][33,5])
for item in range(len(x_train_arr)):
    
    mean_item=np.mean(x_train_arr[item],axis=0)#(1,53)
    std_item=np.std(x_train_arr[item],axis=0)
    np.place(std_item, std_item==0, 1)#if std==0 then change it to 1 for division #std_item[std_item==0]=1 
    x_train_arr[item]=(x_train_arr[item]-mean_item)/std_item

In [5]:
def normalization(X):
    mean=np.mean(X,axis=0)#(1,53)
    std=np.std(X,axis=0)
    np.place(std, std==0, 1)#if std==0 then change it to 1 for division #std_item[std_item==0]=1 
    x_normalized=(X-mean)/std
    return x_normalized,mean,std
def normalization_with_params(X,mean,std):
    return (X-mean)/std

In [6]:
# function to create a list containing mini-batches 
def create_mini_batches(X, y, batch_size): 
    mini_batches = [] 
    data = np.hstack((X, y)) 
    np.random.shuffle(data) 
    n_minibatches = data.shape[0] // batch_size 
    i = 0
  
    for i in range(n_minibatches + 1): 
        mini_batch = data[i * batch_size:(i + 1)*batch_size, :] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        mini_batches.append((X_mini, Y_mini)) 
    if data.shape[0] % batch_size != 0: 
        mini_batch = data[i * batch_size:data.shape[0]] 
        X_mini = mini_batch[:, :-1] 
        Y_mini = mini_batch[:, -1].reshape((-1, 1)) 
        mini_batches.append((X_mini, Y_mini)) 
    return mini_batches 

#что делать с омега0?-просто число но на самом деле вектор одинаковых чисел (8189,1) как и у размерность совпадает
def linear_regression_predict(X,omega,omega0):
    prod=X.dot(omega)#(8190,1)
    #print(np.shape(y+prod))
    return prod+omega0

In [7]:
def root_mean_square_error(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    summands=np.power(y_actual-y_predict,2)
    #print(np.shape(summands))
    mse=np.sum(summands)/len(y_predict)
    return np.sqrt(mse)
   
def r_squared(X,omega,omega0,y_actual):
    y_predict=linear_regression_predict(X,omega,omega0)
    nom=np.sum(np.power(y_actual-y_predict,2))
    denom=np.sum(np.power(y_actual-np.mean(y_actual,axis=0),2))
    return 1-nom/denom

In [8]:
def batch_gradient_descent(X,y_train,learning_rate,num_iterations):
                                   # dim omega
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
    # init omega
    #cost = [0.0 for i in range(num_iterations)]
    it=0
    for it in range(num_iterations): 
        N=len(X)
        error=y_train-linear_regression_predict(X,omega,omega0)
        omega=omega+learning_rate*2*((X.T).dot(error))/N
        omega0=omega0+learning_rate*2*np.sum(error)/N
        cost[it] = r_squared(X,omega,omega0,y_train)
    return omega,omega0,cost

def mini_batch_gradient_descent(X,y_train,learning_rate,num_epoches,batch_size):
                                   # dim omega
    #omega = np.matrix([0.0 for i in range(num_parameters)])
    omega=np.random.sample((X.shape[1],1)) #np.zeros((X.shape[1],1))
    omega0=0
    cost = [] 
    # init omega
    #cost = [0.0 for i in range(num_iterations)]
    it=0
    for it in range(num_epoches): 
        mini_batches = create_mini_batches(X, y_train, batch_size) 
        for mini_batch in mini_batches: 
            X_mini, y_mini = mini_batch 
            N=len(X_mini)
            error=y_mini-linear_regression_predict(X_mini,omega,omega0)
            omega=omega+learning_rate*2*((X_mini.T).dot(error))/N
            omega0=omega0+learning_rate*2*np.sum(error)/N
            cost.append(r_squared(X_mini,omega,omega0, y_mini)) 
    return omega,omega0,cost

In [9]:
r2=[]
rmse=[]
omega=[]
omega0=[]
for item in range(len(x_train_arr)):
    x_test=x_train_arr[item]
    y_test=y_train_arr[item]
    x_train_folds_arr=np.delete(x_train_arr,item)
    y_train_folds_arr=np.delete(y_train_arr,item)
    
    x_train=np.empty((0,x_train_arr[item].shape[1]), int)
    y_train=np.empty((0,y_train_arr[item].shape[1]), int)
    #print(np.shape(y_train_folds_arr))
    for i in range (len(x_train_folds_arr)):
        #print(np.shape(y_train_folds_arr[i]))
        #x_train_all55 =(np.vstack((x_train_all55,x_train_folds_arr[i])))
        x_train=np.concatenate((x_train,x_train_arr[i]))
        y_train=np.concatenate((y_train,y_train_arr[i]))
    #print(np.shape(y_train))
    #x_train,mean,std=normalization(x_train_rd)
    #x_test=normalization_with_params(x_test_rd,mean,std)
    #print(x)
    omega_item,omega0_item,cost=mini_batch_gradient_descent(x_train,y_train,learning_rate = 0.001,num_epoches=10,batch_size=32)
    r2.insert(item,r_squared(x_test,omega_item,omega0_item,y_test))
    rmse.insert(item,root_mean_square_error(x_test,omega_item,omega0_item,y_test))
    omega.insert(item,omega_item)
    omega0.insert(item,omega0_item)
    print(r2[item],rmse[item])

0.34786183806782656 26.552376656623256
0.3716071494018436 27.58937701714877
0.2891103498543576 33.53152181095374
0.22284022796234493 33.149489411266536
0.3055302905763626 26.49727913630891


In [10]:
omega,omega0
rmse,r2

([26.552376656623256,
  27.58937701714877,
  33.53152181095374,
  33.149489411266536,
  26.49727913630891],
 [0.34786183806782656,
  0.3716071494018436,
  0.2891103498543576,
  0.22284022796234493,
  0.3055302905763626])

In [11]:
np.mean(rmse)

29.464008806460242

In [12]:
np.mean(r2)

0.30738997117254707

In [13]:

# initialise data of lists. 
data = {'1':[rmse[0],r2[0],omega0[0], omega[0]], 
        '2':[rmse[1],r2[1],omega0[1], omega[1]],
        '3':[rmse[2],r2[2],omega0[2], omega[2]],
        '4':[rmse[3],r2[3],omega0[3], omega[3]],
        '5':[rmse[4],r2[4],omega0[4], omega[4]],
        'E':[np.mean(rmse),np.mean(r2), np.mean(omega0),np.mean(omega,axis=0)],
        'SD':[np.std(rmse),np.std(r2), np.std(omega0),np.std(omega,axis=0)]} 
 
# Creates pandas DataFrame. 
df = pd.DataFrame(data, index =['RMSE', 'R^2', 'omega0', 'weights']) 
df

,1,2,3,4,5,E,SD
RMSE,26.5524,27.5894,33.5315,33.1495,26.4973,29.464,3.19126
R^2,0.347862,0.371607,0.28911,0.22284,0.30553,0.30739,0.0514744
omega0,7.3802,7.28332,7.44745,7.48059,7.52315,7.42294,0.0840073
weights,"[[0.1156910873320588], [-0.5548558845504826], ...","[[0.12195366168896402], [-0.6228197793398563],...","[[-0.16894189652840144], [-0.7680179195613437]...","[[-0.14455329746851495], [-0.635433489594518],...","[[-0.032010564570510394], [-0.739859838245777]...","[[-0.021572201909280796], [-0.6641973822583955...","[[0.12360584150201263], [0.07873822690077315],..."


In [ ]:
np.mean(omega,axis=0),np.mean(omega0)